In [1]:
# !pip install -r requirements.txt

In [2]:
import pandas as pd
import langdetect
import datetime
import time

import webscrap
import cleanup 

In [3]:
url_ecb_download = "https://www.ecb.europa.eu/press/key/html/downloads.en.html"
url = "https://www.ecb.europa.eu" + webscrap.get_dataset_url(url_ecb_download)

df = pd.read_csv(url, sep = "|", encoding = 'utf-8')

In [4]:
df = cleanup.clean_missing_data(df)
df = cleanup.keep_english(df)
df = cleanup.add_date(df)

date         0
speakers     2
title        0
subtitle     1
contents    39
dtype: int64
            date                  speakers  \
2052  2004-06-21  Gertrude Tumpel-Gugerell   
1415  2009-12-11  Gertrude Tumpel-Gugerell   
2149  2003-05-08    Eugenio Domingo Solans   
425   2017-12-06               Yves Mersch   
223   2019-10-17           Luis de Guindos   

                                                  title  \
2052  Presentation, delivered at the symposium "Zahl...   
1415  The Way Forward with  Monetary, Fiscal and Mac...   
2149                    Catalunya dins l'Europa moderna   
425    Challenges for euro area monetary policy in e...   
223             Speaking notes on climate-related risks   

                                               subtitle  \
2052  Rede von Gertrude Tumpel-Gugerell, Mitglied de...   
1415  Speech by Gertrude Tumpel-Gugerell, Member of ...   
2149  Eugenio Domingo Solans, Membre del Consell de ...   
425   Speech by Yves Mersch, Member of the E

C:\Users\alexj\Git\central-bank-speech-analysis\cleanup.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DayofWeek"][i] = int(datetime.date(df["Year"][i], df["Month"][i], df["Day"][i]).isoweekday())
C:\Users\alexj\Git\central-bank-speech-analysis\cleanup.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DayofWeek_str"][i] = week[int(datetime.date(df["Year"][i], df["Month"][i], df["Day"][i]).weekday())]
C:\Users\alexj\Git\central-bank-speech-analysis\cleanup.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pand

In [9]:
cleanup.todisk(df)
df = cleanup.fromdisk()